# Preprocessing data

## Download dataset

In [8]:
%%bash
pip install kagglehub
pip install opencv-python-headless
pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 53.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 59.8 MB/s eta 0:00:0000:0100:01



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [14]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ikarus777/best-artworks-of-all-time")

print("Path to dataset files:", path)

100%|██████████| 2.29G/2.29G [00:51<00:00, 47.7MB/s]

Extracting files...


Path to dataset files: /Users/atziripe/.cache/kagglehub/datasets/ikarus777/best-artworks-of-all-time/versions/1


In [15]:

%%bash
cd /Users/atziripe/.cache/kagglehub/datasets/ikarus777/best-artworks-of-all-time/versions/1
ls

Cisco Packet Tracer 8.2.2
Desktop
Documents
Downloads
Library
Movies
Music
Pictures
Postman
Postman Agent
Public
go


In [4]:
%%bash
mkdir dataset/
mv /Users/atziripe/.cache/kagglehub/datasets/ikarus777/best-artworks-of-all-time/versions/1 dataset/

In [5]:
#check size of resized image and resize it if needed for the model
import cv2

image = cv2.imread('dataset/1/resized/resized/Alfred_Sisley_60.jpg')

# Get the dimensions of the image
height, width, channels = image.shape

print(f"Width: {width}, Height: {height}, Channels: {channels}")

Width: 528, Height: 640, Channels: 3


In [6]:
#check size of images before resizing 
import cv2

# Load the image
image = cv2.imread('dataset/1/images/images/Alfred_Sisley/Alfred_Sisley_60.jpg')

# Get the dimensions of the image
height, width, channels = image.shape

# Print the dimensions
print(f"Width: {width}, Height: {height}, Channels: {channels}")

Width: 880, Height: 1066, Channels: 3


In [9]:
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import shutil
def resize_images(input_dir, output_dir, target_size=(224, 224)):
    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        if os.path.isdir(class_path):
            output_class_dir = os.path.join(output_dir, class_name)
            os.makedirs(output_class_dir, exist_ok=True)

            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                try:
                    img = Image.open(img_path)
                    img = img.resize(target_size)  # Resize image to target size
                    img.save(os.path.join(output_class_dir, img_name))
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")

# Resize images in the dataset
resize_images("dataset/1/images/images/", "resized_dataset", target_size=(224, 224))


In [11]:
def split_dataset(input_dir, output_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        if os.path.isdir(class_path):
            # List all image paths
            images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.endswith(('.jpg', '.png'))]

            # Split into train, validation, and test sets
            train, temp = train_test_split(images, test_size=(1 - train_ratio), random_state=42)
            val, test = train_test_split(temp, test_size=(test_ratio / (val_ratio + test_ratio)), random_state=42)

            # Create output directories and move files
            for split_name, split_data in zip(['train', 'validation', 'test'], [train, val, test]):
                split_dir = os.path.join(output_dir, split_name, class_name)
                os.makedirs(split_dir, exist_ok=True)

                for img_path in split_data:
                    shutil.copy(img_path, split_dir)

# Split the resized dataset
split_dataset("resized_dataset", "final_dataset")
